In [5]:
1+1

2

In [6]:
import os
os.chdir("../")

In [15]:
os.chdir("TextSummerizer")

In [16]:
%pwd

'd:\\lokesh\\DS\\MLops\\Projects\\TextSummerizer'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir:Path
    data_path:Path
    tokenizer_name:Path

In [20]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.commomn import read_yaml,create_directories

In [22]:
class ConfigurationManager:
    def __init__(self,
                 config_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [23]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

d:\lokesh\DS\MLops\Projects\TextSummerizer\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-15 18:35:03,748: INFO: config: PyTorch version 2.6.0 available.]


In [28]:
class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_example_to_features(self,example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return (
            {
                'input_ids' : input_encoding['input_ids'],
                'attention_mask': input_encoding['attention_mask'],
                'labels': target_encoding['input_ids']
            }
        )
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_example_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [29]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config = data_transformation_config)
data_transformation.convert()

[2025-04-15 18:44:27,515: INFO: commomn: yaml file: config\config.yaml loaded successfully]
[2025-04-15 18:44:27,519: INFO: commomn: yaml file: params.yaml loaded successfully]
[2025-04-15 18:44:27,522: INFO: commomn: created directories at: artifacts]
[2025-04-15 18:44:27,525: INFO: commomn: created directories at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]d:\lokesh\DS\MLops\Projects\TextSummerizer\venv\lib\site-packages\transformers\tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 78904.85 examples/s]
